In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:


# write all code in one cell

# ========================Load data=========================
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
import pickle


with open('/content/drive/My Drive/retrain_dataset_to_remove_space_no_blog','rb') as f:#  new_number_and_words_classification new_seqtoseq_classification
    train_texts,train_output = pickle.load(f)



# convert string to lower case
# train_texts = train_df[1].values
# for  m,i in enumerate(train_texts):
#     if i == ' ' or i == '' or i == '  ':
#       del train_texts[m]
#       del train_output[m]
# for  m,i in enumerate(train_texts_validation):
#     if i == ' ' or i == '' or i == '  ':
#       del train_texts_validation[m]
#       del train_output_validation[m]

      # train_texts = [s.lower() for s in train_texts]

# # test_texts = test_df[1].values
# train_texts_validation = [s[0].lower() for s in train_texts_validation]

# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
# tkw =Tokenizer(num_words=None, oov_token='UNK')
# tkw.fit_on_texts(train_output)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

# -----------------------Skip part start--------------------------
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789 "
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
# tkw.word_index[' '] = len(tkw.word_index) + 2
# -----------------------Skip part end----------------------------

# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
train_sequences_output = tk.texts_to_sequences(train_output)

# train_sequences_validation = tk.texts_to_sequences(train_texts_validation)
#Padding
train_data = pad_sequences(train_sequences, maxlen=60, padding='post')
train_data_output = pad_sequences(train_sequences_output, maxlen=60, padding='post')

# train_data_validation = pad_sequences(train_sequences_validation, maxlen=90, padding='post')
# for  k,i in enumerate(train_output):
#   print(i)
# train_output = pad_sequences(train_output,maxlen=87,padding='post')
#   train_output[k] = data
# for  k,i in enumerate(train_output_validation):
# train_output_validation = pad_sequences(train_output_validation,maxlen=90,padding='post')
#   train_output_validation[k] = data

# test_data = pad_sequences(test_texts, maxlen=50, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
train_classes = np.array(train_data_output, dtype='float32')
# train_output_3 = np.array(train_output_3, dtype='float32')
# train_data_validation = np.array(train_data_validation, dtype='float32')
# train_output_validation = np.array(train_output_validation, dtype='float32')

# =======================Get classes================
# train_classes = train_df[0].values
# train_class_list = [x - 1 for x in train_output]

# test_classes = test_df[0].values
# test_class_list = [x - 1 for x in test_classes]

from keras.utils import to_categorical

train_classes = to_categorical(train_data_output)
# train_classes_1 = to_categorical(train_output_3)
# train_classes_validation = to_categorical(train_output_validation)
# test_classes = to_categorical(test_class_list)





In [0]:
# =====================Char CNN=======================
# parameter
from keras.layers import Concatenate,Add,Bidirectional,LSTM,TimeDistributed,GRU,BatchNormalization,CuDNNLSTM
from keras.utils import plot_model

input_size = 60
vocab_size = len(tk.word_index)
embedding_size = 69
conv_layers = [[256, 10],
               [256, 10],
               [256, 7],
               [256, 7],
               [256, 7],
               [256, 3],
               [256, 3],]
fully_connected_layers = [200, 200]
# num_of_classes = 70
dropout_p = 0.5
optimizer = 'Nadam'
loss = 'categorical_crossentropy'

# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            300,
                            input_length=input_size,
                            mask_zero=False)
#                             weights=[embedding_weights])

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# x = Position_Embedding()(x)
# x = Attention(8, 10)([x, x, x])

# x1 = Flatten()(x)
# predictions_1 = Dense(num_of_classes,activation='softmax')(x1)

# from keras_pos_embd import PositionEmbedding
# position_embedding = PositionEmbedding(
#     input_shape=(None,),
#     input_dim=90,     # The maximum absolute value of positions.
#     output_dim=embedding_size,     # The dimension of embeddings.  # The index that presents padding (because `0` will be used in relative positioning).
#     name='Pos-Embd',
# )
# xp = position_embedding(inputs)
# Conv
# for filter_num, filter_size in conv_layers:
#     if filter_size == 10:
# #       if k == 0:
#         x_10 = Conv1D(filter_num, filter_size)(x)
#         x_10 = Activation('relu')(x_10)
# #         k = 1
# #       else:
# #         x_10 = Conv1D(filter_num, filter_size)(x_10)
# #         x_10 = Activation('relu')(x_10)
#     elif filter_size == 7:
# #       if d == 0:
#         x_7 = Conv1D(filter_num, filter_size)(x)
#         x_7 = Activation('relu')(x_7)
# #         d = 1
# #       else:
# #         x_7 = Conv1D(filter_num, filter_size)(x_7)
# #         x_7 = Activation('relu')(x_7)
#     elif filter_size == 5:
# #       if v == 0:
#         x_5 = Conv1D(filter_num, filter_size)(x)
#         x_5 = Activation('relu')(x_5)
# #         v = 1
# #       else:
# #         x_5 = Conv1D(filter_num, filter_size)(x_5)
# #         x_5 = Activation('relu')(x_5)
#     elif filter_size == 3:
# #       if b == 0:
#         x_3 = Conv1D(filter_num, filter_size)(x)
#         x_3 = Activation('relu')(x_3)
# #         b = 1
# #       else:
# #         x_3 = Conv1D(filter_num, filter_size)(x)
# #         x_3 = Activation('relu')(x_3)
        
# x = Add()([x_10,x_7,x_5,x_3])
# conv_layers = [[256,10],[256,10],[256,7],[256,7],[256,7],[256,3],[256,3]]
# for filter_num, filter_size in conv_layers:
# #     if filter_size == 10:
#       x = Conv1D(filter_num, filter_size,padding='same')(x)
#       x = Activation('elu')(x)

# for filter_num, filter_size in conv_layers:
#     if filter_size == 10:
conv = Conv1D(100, 60,padding='same')(x)
conv = BatchNormalization()(conv)
conv = Activation('elu')(conv)
conv = Dropout(0.3)(conv)
# conv = MaxPooling1D(2,padding='valid')(conv)
conv = Conv1D(50, 60,padding='same')(conv)
conv = BatchNormalization()(conv)
conv = Activation('elu')(conv)
conv = Dropout(0.3)(conv)
# conv = MaxPooling1D(2,padding='valid')(conv)
# conv = Add()([conv,x])
# conv = Conv1D(256, 70,padding='same')(conv)
# conv = BatchNormalization()(conv)
# conv = Activation('elu')(conv)
# conv = Dropout(0.3)(conv)
# # conv = Add()([conv,x])
# # conv = MaxPooling1D(2,strides=3,padding='valid')(conv)
# conv = Conv1D(100, 70,padding='same')(conv)
# conv = Activation('elu')(conv)
# conv = Conv1D(100, 90,padding='same')(conv)
# conv = Activation('elu')(conv)
# bilstm = Bidirectional(CuDNNLSTM(3,return_sequences=True,))(x)
# bilstm_flatten = Flatten()(x)  # (None, 8704)
# conv_flatten = Flatten()(conv)  # (None, 8704)

x = Concatenate()([conv,x])
#     if pooling_size != -1:
#         x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
# conv_flatten = Flatten()(conv)  # (None, 8704)
# 
x = Bidirectional(CuDNNLSTM(5,return_sequences=True))(x)
# encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder
# state_h = Add()([forward_h, backward_h])
# state_c = Add()([forward_c, backward_c])
# encoder_last = [state_h, state_c]



#     if pooling_size != -1:
#         x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
# bilstm_flatten = Flatten()(encoder)  # (None, 8704)
# 
# x = Concatenate()([conv_flatten,bilstm_flatten])

# x.shape
# x = LSTM(1,)(x)
# # Fully connected layers
# for dense_size in fully_connected_layers:
# x_2 = Dense(50,activation='elu')(x)
# x = Flatten()(conv)
# x = TimeDistributed(Dense(100, activation='elu'))(x)  # dense_size == 1024
# x = TimeDistributed(Dropout(dropout_p))(x)
# x = TimeDistributed(Dense(100, activation='relu'))(x)  # dense_size == 1024
# x = TimeDistributed(Dropout(dropout_p))(x)

# # Output Layer
# d = len(tkw.word_index)
predictions = TimeDistributed(Dense(38, activation='softmax'))(x)#

# print(predictions.shape)
# # Build model
model = Model(inputs=inputs, outputs=[predictions])#
model.compile(optimizer=optimizer, loss=loss, metrics=['categorical_accuracy'])  # Adam, categorical_crossentropy
model.summary()

# # # 1000 training samples and 100 testing samples
# # indices = np.arange(train_data.shape[0])
# # np.random.shuffle(indices)
# #
# x_train = train_data[indices][:1000]
# y_train = train_classes[indices][:1000]
#
# # x_test = test_data[:100]
# # y_test = test_classes[:100]

indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]
# y_train_1 = train_classes_1[indices]
# plot_model(model, to_file='/content/drive/My Drive/singewordvisualmodel.png')

# Training


In [0]:
model.fit(x=x_train, y=[y_train],#predictions_1
          validation_split = 0.01,
#           validation_data=(x_valid,y_valid),
          batch_size=100,
          epochs=10,shuffle=True,)#callbacks=callbacks_list)

In [0]:
# prediction

In [0]:
datas = model.predict(train_data,verbose=1) # test = model.predict(data(['hidufitler']))

In [0]:
#check the predicted data

In [0]:
whole = []
for  i in datas:
  lists = []
  for  j in i:
    lists.append(np.argmax(j))
  whole.append(lists)

reverse_dict = {}
for  i in char_dict.items():
  reverse_dict[i[1]] = i[0]

whole_words =[]
for  k in whole:
  lists = []
  for j in k:
    if j:
      lists.append(reverse_dict[j])
#   print(lists)
  whole_words.append(''.join(lists))

In [0]:
for i in zip(train_texts[120000:],train_output[120000:],whole_words[120000:]):
  print(i)

In [0]:
try:
    model_json = model.to_json()
    with open("/content/drive/My Drive/classification_words_new_space_model_3.json", "w") as json_file:
        json_file.write(model_json)
except:
    pass
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/classification_words_new_space_model_3.h5")
print("Saved model to disk")